In [1]:
import pandas as pd 
import numpy as np 
import math
import pandas as pd
import sys
from apyori import apriori
from tqdm import tqdm
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from datetime import datetime

In [2]:
class Error(Exception):
    def __str__(self):
        return "허용되지 않는 행동입니다."

In [3]:
print(sys.path[0])

/home/shinjk1156/Project_code/Retail_Project/Code


### Setting Grid World

In [4]:
df = pd.read_excel('/home/shinjk1156/Project_code/Retail_Project/Data/RFID_IPS_데이터추출_20200904.xlsx', sheet_name= None)

In [5]:
grid_dic = {"청과" : [(0,1),(0,2),(0,3),(0,4)], "곡물" : [(1,1),(1,2),(1,3),(1,4)], "입구" : [(0,0)], 
            "채소" : [(0,5),(0,6),(0,7),(1,5),(1,6),(1,7)], "닭고기" : [(0,8)], "수산" : [(0,9), (1,8), (1,9)], "축산" : [(0,10), (1,10)], 
            "델리카" : [(0,11), (1,11)], "계산대" : [(1,0),(2,0),(3,0),(4,0)], "청소욕실" : [(2,5),(2,6),(3,5),(3,6)],
            "주방용품" : [(2,7),(3,7)], "언더웨어" : [(4,5), (4,6)], "H&B" : [(2,1),(2,2),(2,3),(2,4), (3,1),(3,2),(3,3),(3,4), (4,1),(4,2),(4,3),(4,4)],
            "건강,차" : [(4,7)], "조미식품" : [(2,8), (2,9)], "인스턴트" : [(3,8), (3,9)], "과자" : [(4,8), (4,9)], 
            "냉장냉동" : [(2,10), (2,11)], "데일리" : [(3,10), (3,11)], "음료,주류" : [(4,10),(4,11)],
            "스포츠" : [(5,1)], "문구" : [(5,2)], "완구" : [(5,3)], "자동차" : [(5,4)], "애완원예용품" : [(5,5)], "인테리어" : [(5,6)],
            "수예" : [(5,7)], "세제" : [(5,8)], "위생용품" : [(5,9)], "금지구역" : [(5,10),(5,11),(5,0)]}

In [6]:
class Agent:
    def __init__(self, initial_position):
        self.action_space = np.array([(0,0), (-1,0), (1,0), (0,-1), (0,1)])
        self.action_pr = np.array([0.20,0.20,0.20,0.20,0.20])
        self.pos = np.array(initial_position)

    def set_pos(self, position):
        self.pos = position
        return self.pos

    def get_pos(self):
        return self.pos

In [7]:
class Env:
    # action space = [(0,0)), (-1,0)), (1,0)), (0,-1)), (0,1)]
    def __init__(self, grid_dic):
        self.grid_dic = grid_dic
        self.grid_world = self.make_grid_world()

    def make_grid_world(self):
        grid_world = pd.DataFrame(np.zeros((6,12)))
        for key in self.grid_dic.keys():
            for x,y in self.grid_dic[key]:
                grid_world.iloc[x,y] = key
        return grid_world

    def move(self, agent, action):
        terminal = False
        reward = 0
        #현재 좌표
        current_pos = agent.pos
        #현재 좌표에 대응되는 현재 매대
        cur_key = self.grid_world.iloc[current_pos[0], current_pos[1]]
        #이동된 좌표
        try: next_pos = agent.pos + agent.action_space[action] #아직은 전이확률을 고려하지 않음
        except: raise Error()
        #이동된 좌표에 대응되는 다음 매대
        try: next_key = self.grid_world.iloc[next_pos[0], next_pos[1]]
        except: next_key = "금지구역"

        #매대간 이동이 존재하는대도 만약 cur_key와 next_key가 같을 시,(이동이 발생했는대도 그리드월드 설계문제로 인해 매대 이동이 발생하지 못함)
        #이 때 이동좌표를 변경시켜줌
        if action != 0:
            if cur_key == next_key:
                temp = np.array(self.grid_dic[next_key])
                if action == 1: next_pos[0] = min(temp.T[0]) - 1
                elif action == 2: next_pos[0] = max(temp.T[0]) + 1
                elif action == 3: next_pos[1] = min(temp.T[1]) - 1
                elif action == 4: next_pos[1] = max(temp.T[1]) + 1
                else: raise Error()
        #현재 좌표가 최종목적지인지 확인
        if self.grid_world.iloc[current_pos[0], current_pos[1]] == "계산대":
            terminal = True
            show_next_pos = agent.set_pos(agent.pos)
            #reward = 0

        #이동된 좌표가 이동 불가능한 지점인지 확인
        elif next_key == "금지구역" or tuple(next_pos) in self.grid_dic["금지구역"] or next_pos[0] < 0 or next_pos[1] < 0:
            terminal = True
            show_next_pos = agent.set_pos(agent.pos)
            #reward = 0

        #이동가능한 동선이면 '이동':
        else:
            show_next_pos = agent.set_pos(next_pos)
        
        return show_next_pos, reward, terminal



In [8]:
agent = Agent((1,1)) 
env = Env(grid_dic)
print(env.move(agent, 4))
agent.get_pos()

(array([1, 5]), 0, False)


array([1, 5])

In [9]:
env.grid_world

,0,1,2,3,4,5,6,7,8,9,10,11
0,입구,청과,청과,청과,청과,채소,채소,채소,닭고기,수산,축산,델리카
1,계산대,곡물,곡물,곡물,곡물,채소,채소,채소,수산,수산,축산,델리카
2,계산대,H&B,H&B,H&B,H&B,청소욕실,청소욕실,주방용품,조미식품,조미식품,냉장냉동,냉장냉동
3,계산대,H&B,H&B,H&B,H&B,청소욕실,청소욕실,주방용품,인스턴트,인스턴트,데일리,데일리
4,계산대,H&B,H&B,H&B,H&B,언더웨어,언더웨어,"건강,차",과자,과자,"음료,주류","음료,주류"
5,금지구역,스포츠,문구,완구,자동차,애완원예용품,인테리어,수예,세제,위생용품,금지구역,금지구역


### Association Rule Mining

In [96]:
pos_detail = df['POS_DETAIL']
pog_cate = df["POG-카테고리"]
pog = df["POG"]
info_item = df["상품정보"]
info_cate = df['상품-카테고리']

In [97]:
pog_df = pd.merge(pog_cate, pog, on = "POG 그룹 ID", how = "left")
pog_df.rename(columns = {"DIVISION" : "DIV_CD", "DEPT" : "DEPT_CD", "POG 이름_x" : "POG 이름", "POG 이름_y":"POG 그룹 이름"}, inplace = True)
pog_df.drop(index = 0, inplace = True)
pog_df.drop(columns = ["사용 유무", "POG 그룹 ID", "POG ID","POG 이름"], inplace = True)
pog_df.drop_duplicates(inplace = True)

In [98]:
#info_item.drop(index = 0, inplace = True)
info_item.rename(columns = {"DIVISION CODE":"DIV_CD", "DEPT CODE" : "DEPT_CD", "SECTION CODE" : "SECTION_CD", "CLASS CODE" : "CLASS_CD"}, inplace= True)
info_df = pd.merge(info_item, info_cate, on = ["DIV_CD", "DEPT_CD", "SECTION_CD", "CLASS_CD"], how = "left")
info_df.drop(columns= ["바코드번호", "판매 가격"], inplace= True)
pog_in_df = pd.merge(info_df, pog_df, on = ["DIV_CD", "DEPT_CD"], how = "left")
item_df = pd.merge(pos_detail.loc[:,["쇼핑 ID", "자체상품코드"]], pog_in_df, on = "자체상품코드", how = "left")
item_df.rename(columns= {"DIVISION CODE" : "DIVISION", "DEPT CODE" : "DEPT"}, inplace = True)
item_df.drop(index= 0, inplace = True)
item_df.drop_duplicates(inplace = True)
item_df.reset_index(drop= True, inplace = True)

In [99]:
item_df

,쇼핑 ID,자체상품코드,상품명,DIV_CD,DEPT_CD,SECTION_CD,CLASS_CD,DIV_NAME,DEPT_NAME,SECTION_NAME,CLASS_NAME,POG 그룹 이름
0,1,1355218000,●동원 참치세트 16호,27,185,1617,11400,인스턴트,선물세트,통조림선물세트,참치통조림혼합선물세트,NaN
1,1,0002960000,피홍합,20,154,1452,13770,생선,해물연체류,각패류,홍합,NaN
2,1,1289860000,다이소2000원균일가제품,35,782,7238,67347,주방용품,주방시즌/균일가,주방균일가,주방용품균일가,NaN
3,1,1260181000,와이즐렉 네임펜흑3입,39,256,1849,12020,문구,필기도구,펜,유성매직,문구
4,1,1248045000,와이즐렉스테플러10호세트,39,258,1860,12072,문구,일반사무용품,OA용품,스테플러,문구
...,...,...,...,...,...,...,...,...,...,...,...,...
356082,26043,1480971000,풀)국산 더단단한 부침두부,19,146,7250,67537,채소,두부,풀무원두부,풀무원포장두부,채소2
356083,26043,1494164000,아침의 계란(왕란/15입),72,805,7378,67992,닭고기,계란,일반란,소포장란,닭고기
356084,26045,0008750000,친환경 쌈채소,19,149,1391,10624,채소,친환경채소,친환경쌈채류,친환경쌈채류,채소2
356085,26045,0030732000,바지락살,20,154,1453,10751,생선,해물연체류,살패류,바지락살,NaN


In [100]:
#DIV_NAME들을 구분하여 GW카테고리에 맞춰 새로운 칼럼생성
gw_list = []
for i in range(len(item_df)):
    name = item_df.loc[i, "DIV_NAME"]
    if name == "과일":
        gw_list.append("청과")
    elif name in ["생선","건해산물"]:
        gw_list.append("수산")
    elif name in ["돼지고기" ,"소고기"]:
        gw_list.append("축산")
    elif name == "조리식품":
        gw_list.append("델리카")
    elif name in ["H&B", "섬유잡화", "장신잡화", "남성의류편집", "피혁잡화", "신발", "아동유아편집"]:
        gw_list.append("H&B")
    elif name in ["음료", "주류"]:
        gw_list.append("음료,주류")
    elif name in ["일반스포츠", "레져스포츠", "스포츠NB"]:
        gw_list.append("스포츠")
    else:
        gw_list.append(name)
item_df["GW_Cate"] = gw_list
item_df.drop(columns = ["POG 그룹 이름", "DEPT_NAME","SECTION_NAME", "CLASS_NAME"], inplace= True)
item_df.drop_duplicates(inplace = True)
item_df.dropna(inplace = True)
item_df.reset_index(drop = True, inplace= True)

In [102]:
item_df.GW_Cate.unique()

array(['인스턴트', '수산', '주방용품', '문구', '과자', '채소', '음료,주류', '청과', '위생용품',
       '냉장냉동', '세제', '데일리', '축산', '조미식품', 'H&B', '닭고기', '건강,차', '곡물',
       '델리카', '청소욕실', '애완원예용품', '인테리어', '언더웨어', '자동차', '완구', '스포츠', '수예'],
      dtype=object)

In [33]:
shid_list = item_df["쇼핑 ID"].unique()
transaction_list = []
for s_id in tqdm(shid_list, desc = "Create Transaction_list"):
    transaction_list.append(list(item_df[item_df["쇼핑 ID"] == s_id].GW_Cate.unique()))


100%|██████████| 13864/13864 [02:14<00:00, 102.79it/s]


In [34]:
t_encoder = TransactionEncoder()
t_array = t_encoder.fit(transaction_list).transform(transaction_list)
t_df = pd.DataFrame(t_array, columns= t_encoder.columns_)

In [53]:
fre_itemset = apriori(t_df, min_support= 0.05, use_colnames= True)
fre_itemset.sort_values(by = "support", ascending = False, inplace = True)
fre_itemset.reset_index(drop = True, inplace= True)
fre_itemset

,support,itemsets
0,0.707372,(채소)
1,0.569749,(데일리)
2,0.462709,(청과)
3,0.452178,(수산)
4,0.419432,"(데일리, 채소)"
...,...,...
693,0.050202,"(인스턴트, 수산, 주방용품, 채소)"
694,0.050130,"(인스턴트, 데일리, 델리카, 채소, 청과)"
695,0.050058,"(청소욕실, 데일리, 냉장냉동)"
696,0.050058,"(인스턴트, H&B, 냉장냉동)"


In [61]:
import csv

In [63]:
with open("..rules.csv", 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerows(item_list)

In [59]:
min_num_items = 5
num_collect = 10
i = 0
count = 0
item_list=[]
while(True):
    if count == num_collect:
        break
    if len(fre_itemset.itemsets[i]) >= min_num_items:
        count += 1
        item_list.append(list(fre_itemset.itemsets[i]))
    i+=1

return item_list


In [37]:
association_rules(fre_itemset, metric="confidence", min_threshold=0.3) 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(H&B),(과자),0.196336,0.418782,0.098745,0.502939,1.200955,0.016523,1.169308
1,(H&B),(냉장냉동),0.196336,0.383078,0.086267,0.439383,1.146979,0.011055,1.100433
2,(H&B),(닭고기),0.196336,0.261829,0.059290,0.301984,1.153362,0.007884,1.057527
3,(H&B),(데일리),0.196336,0.569749,0.126010,0.641807,1.126474,0.014148,1.201173
4,(H&B),(델리카),0.196336,0.333309,0.070326,0.358193,1.074655,0.004885,1.038771
...,...,...,...,...,...,...,...,...,...
3499,"(조미식품, 데일리, 채소)","(인스턴트, 청과)",0.147144,0.192297,0.055251,0.375490,1.952662,0.026956,1.293340
3500,"(조미식품, 청과, 데일리)","(인스턴트, 채소)",0.101125,0.291474,0.055251,0.546362,1.874478,0.025776,1.561876
3501,"(조미식품, 청과, 채소)","(인스턴트, 데일리)",0.128318,0.264786,0.055251,0.430579,1.626136,0.021274,1.291160
3502,"(인스턴트, 조미식품)","(청과, 데일리, 채소)",0.162074,0.240118,0.055251,0.340899,1.419713,0.016334,1.152906


### Sequence Pattern Mining

In [5]:
df = pd.read_excel('/home/shinjk1156/Project_code/Retail_Project/Data/RFID_IPS_데이터추출_20200904.xlsx', sheet_name= None)
line = df['라인정보']
repeater = df["리피터 마스터"]
pog = df['POG']
pog_cate = df['POG-카테고리']
shoping = df['가공된 쇼핑 데이터']
#pog.drop(index= 0, inplace = True)
#repeater.drop(index= 0, inplace = True)
pog["POG 그룹 ID"] = pog["POG 그룹 ID"].map(str)
pog_cate["POG 그룹 ID"] = pog_cate["POG 그룹 ID"].map(str)
moving1 = pd.read_csv("/home/shinjk1156/Project_code/Retail_Project/Data/실시간저장데이터_20110208_20110220.txt", sep = "\t", header = None)
moving2 = pd.read_csv("/home/shinjk1156/Project_code/Retail_Project/Data/실시간저장데이터_20110117_20110207.txt", sep = "\t", header = None)
moving = moving1.append(moving2)
moving.columns = ["SINK ID", "태그 ID","TAG 강도", "리피터 ID", "저장시간"]

repeater["리피터 ID"] = repeater["리피터 ID"].astype(str)
moving["리피터 ID"] = moving["리피터 ID"].astype(str)

mer_df = pd.merge(repeater, pog, on = "POG 그룹 ID", how = "left")

In [116]:
mer_df


,리피터 ID,X 좌표,Y 좌표,POG 그룹 ID,스타트 여부,POS 여부,사용 강도,사용여부,상품 여부,가중치,리포트에 보여줄지에 대한 여부,POG 이름
0,REPEATER_ID,X_Coord,Y_Coord,POG_GROUP_ID,IS_START,IS_END,USE_STRENGTH,USE_YN,ITEM_SALE_YN,POINT_LEVEL,REPORT_USE_YN,POG_GROUP_DESC
1,1001,50,568,00000,Y,N,0,Y,N,3,N,NaN
2,1002,50,593,00000,Y,N,0,Y,N,3,N,NaN
3,1003,149,556,11101,N,N,0,Y,Y,3,Y,청과
4,1004,150,593,11101,N,N,0,Y,Y,3,Y,청과
...,...,...,...,...,...,...,...,...,...,...,...,...
170,1302,122,279,23001,N,N,0,Y,Y,3,N,H&B
171,1303,101,138,25301,N,N,0,Y,Y,3,N,문구
172,1304,173,279,47001,N,N,0,Y,Y,3,N,잡화
173,1305,213,209,51001,N,N,0,Y,Y,3,N,언더웨어


In [114]:
moving

,SINK ID,태그 ID,TAG 강도,리피터 ID,저장시간
0,142,047,1,1054,2011-02-08 01:58:23.610
1,150,047,1,1151,2011-02-08 01:58:32.483
2,150,047,0,1047,2011-02-08 01:58:34.110
3,150,047,1,1047,2011-02-08 01:58:34.657
4,150,047,1,1047,2011-02-08 01:58:35.483
...,...,...,...,...,...
5051407,152,52,1,1168,2011-02-07 23:56:31.547
5051408,155,52,1,1168,2011-02-07 23:56:31.670
5051409,152,52,1,1168,2011-02-07 23:56:36.703
5051410,152,52,1,1168,2011-02-07 23:56:39.767


In [392]:
to_shop = pd.DataFrame()
# this process take many time,
for i in tqdm(range(1,len(shoping))):
    sample = moving[(moving.저장시간 >= shoping.loc[i,"쇼핑 시작시간"]) & (moving.저장시간 <= shoping.loc[i,"쇼핑 종료 시간"]) & (moving["태그 ID"] ==shoping.loc[i, "태그 ID"])].copy()
    if len(sample) == 0:
        pass
    else:
        sample.sort_values(by = "저장시간", inplace = True)
        sample = pd.merge(sample, repeater, on = "리피터 ID", how = "left")
        sample["쇼핑 ID"] = [shoping.loc[i, "쇼핑 ID"]]*len(sample)
        to_shop = to_shop.append(sample)
#to_shop.to_csv("동선_전처리_0222_0220.csv", index = False)


  0%|          | 2/13865 [00:03<7:10:58,  1.87s/it]


KeyboardInterrupt: 

In [542]:
fdf = pd.read_csv("/home/shinjk1156/Project_code/Retail_Project/Code/Preprocessing/Data/동선_전처리_0122_0220_final.csv")

In [543]:
fdf

,SINK ID,태그 ID,TAG 강도,리피터 ID,저장시간,X 좌표,Y 좌표,POG 그룹 ID,스타트 여부,POS 여부,사용 강도,사용여부,상품 여부,가중치,리포트에 보여줄지에 대한 여부,쇼핑 ID
0,157,1,1,1001,2011-01-22 15:04:43.467,50.0,568.0,0.0,Y,N,0.0,Y,N,3.0,N,1
1,153,1,1,1006,2011-01-22 15:04:50.297,108.0,593.0,11101.0,N,N,0.0,Y,Y,3.0,Y,1
2,153,1,1,1006,2011-01-22 15:04:51.467,108.0,593.0,11101.0,N,N,0.0,Y,Y,3.0,Y,1
3,156,1,1,1005,2011-01-22 15:04:53.420,182.0,593.0,14601.0,N,N,0.0,Y,Y,3.0,Y,1
4,152,1,1,1022,2011-01-22 15:05:01.517,213.0,593.0,14601.0,N,N,0.0,Y,Y,3.0,Y,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4148213,141,200,1,1205,2011-02-20 14:47:15.110,31.0,235.0,1.0,N,Y,0.0,Y,N,3.0,N,26045
4148214,141,200,1,1205,2011-02-20 14:47:51.093,31.0,235.0,1.0,N,Y,0.0,Y,N,3.0,N,26045
4148215,141,200,1,1205,2011-02-20 14:47:52.187,31.0,235.0,1.0,N,Y,0.0,Y,N,3.0,N,26045
4148216,141,200,1,1205,2011-02-20 14:47:53.170,31.0,235.0,1.0,N,Y,0.0,Y,N,3.0,N,26045


In [544]:
#fdf = to_shop.copy()
fdf.drop(columns = ["사용 강도", "상품 여부","가중치","리포트에 보여줄지에 대한 여부","스타트 여부","사용여부","POS 여부", "TAG 강도"], inplace = True)
fdf.dropna(inplace = True)

In [545]:
info_item = df["상품정보"]
info_cate = df["상품-카테고리"]
info_item.rename(columns = {"DIVISION CODE":"DIV_CD", "DEPT CODE" : "DEPT_CD", "SECTION CODE" : "SECTION_CD", "CLASS CODE" : "CLASS_CD"}, inplace= True)
info_df = pd.merge(info_item, info_cate, on = ["DIV_CD", "DEPT_CD", "SECTION_CD", "CLASS_CD"], how = "left")
info_df.drop(columns= ["바코드번호", "판매 가격"],index = 0 ,inplace= True)
info_df

,자체상품코드,상품명,DIV_CD,DEPT_CD,SECTION_CD,CLASS_CD,DIV_NAME,DEPT_NAME,SECTION_NAME,CLASS_NAME
1,0000023000,부사대표,17,132,1262,10296,과일,국산과일,사과,부사
2,0000130000,한우정육(국내산2등급),71,800,7353,67861,소고기,우육,한우,한우정육
3,0000222000,종가집나박김치,23,818,7448,68379,조리식품,특정반찬,김치,일반김치
4,0000312000,햇사과대표,17,132,1262,20001,과일,국산과일,사과,햇사과
5,0000335000,고산지 바나나,17,133,1285,67166,과일,수입과일,바나나,프리미엄바나나
...,...,...,...,...,...,...,...,...,...,...
538636,9900010000,롯데백화점PP카드일십만원권,99,904,9005,90008,기타,P.P 카드,자사 P.P 카드,롯데백화점P.P 카드
538637,9900011000,롯데예식장(예식),99,905,9006,90009,기타,기타,기타,기타
538638,9900013000,엘지주유권50000,99,903,9003,90006,기타,상품권,타사상품권,타사상품권
538639,9900052000,계란(할인),72,805,7378,67991,닭고기,계란,일반란,판란


In [547]:

pog["POG 그룹 ID"] = pog["POG 그룹 ID"].map(str)
pog_cate["POG 그룹 ID"] = pog_cate["POG 그룹 ID"].map(str)
pog_df = pd.merge(pog_cate, pog, on="POG 그룹 ID", how = "left")
pog_df.rename(columns = {"DIVISION" : "DIV_CD", "DEPT" : "DEPT_CD", "POG 이름_x" : "POG 이름", "POG 이름_y":"POG 그룹 이름"}, inplace = True)
pog_df.drop(index = 0, inplace = True)
pog_df.drop(columns = ["사용 유무", "POG ID","POG 이름","DEPT_CD"], inplace = True)
pog_df.drop_duplicates(inplace = True)
pog_df

,POG 그룹 ID,DIV_CD,POG 그룹 이름
1,10101,29,과자
2,10102,32,세제
3,10103,33,위생용품
4,11101,17,청과
5,11201,19,채소
...,...,...,...
406,79603,65,홈/콘/모나카 아이스크림
407,80501,72,닭고기
427,81601,21,"미역,다시마"
429,82602,38,수예


In [548]:
pog_in_df = pd.merge( pog_df, info_df.loc[:,["DIV_CD", "DIV_NAME"]], on = ["DIV_CD"], how = "left")

In [549]:
pog_in_df.drop(columns = ["POG 그룹 이름"], inplace = True)
pog_in_df.drop_duplicates(inplace = True)
pog_in_df.reset_index(drop = True, inplace = True)

In [550]:
pog_in_df

,POG 그룹 ID,DIV_CD,DIV_NAME
0,10101,29,과자
1,10102,32,세제
2,10103,33,위생용품
3,11101,17,과일
4,11201,19,채소
...,...,...,...
113,79603,65,냉장냉동
114,80501,72,닭고기
115,81601,21,건해산물
116,82602,38,수예


In [551]:
#DIV_NAME들을 구분하여 GW카테고리에 맞춰 새로운 칼럼생성
gw_list = []
for i in range(len(pog_in_df)):
    name = pog_in_df.loc[i, "DIV_NAME"]
    if name == "과일":
        gw_list.append("청과")
    elif name in ["생선","건해산물"]:
        gw_list.append("수산")
    elif name in ["돼지고기" ,"소고기"]:
        gw_list.append("축산")
    elif name == "조리식품":
        gw_list.append("델리카")
    elif name in ["H&B", "섬유잡화", "장신잡화", "남성의류편집", "피혁잡화", "신발", "이지캐주얼" , "아동유아편집","여성의류편집","여성의류NB","남성의류NB","아동유아NB"]:
        gw_list.append("H&B")
    elif name in ["음료", "주류"]:
        gw_list.append("음료,주류")
    elif name in ["일반스포츠", "레져스포츠", "스포츠NB"]:
        gw_list.append("스포츠")
    else:
        gw_list.append(name)

In [552]:
pog_in_df["GW_Cate"] = gw_list
pog_in_df.drop(columns = ["DIV_CD", "DIV_NAME"],inplace = True)
pog_in_df.drop_duplicates(inplace = True)

In [553]:
pog_id = []
gw_ca = []
for i in range(17):
    if i == 0:
        pog_id.append(0)
        gw_ca.append("입구")
    else:
        pog_id.append(i)
        gw_ca.append("계산대")
io_df = pd.DataFrame()
io_df["POG 그룹 ID"] = pog_id
io_df["GW_Cate"] = gw_ca

In [554]:
pog_in_df = pd.concat([pog_in_df, io_df])
pog_in_df["POG 그룹 ID"] = pog_in_df["POG 그룹 ID"].astype(int)
fdf["POG 그룹 ID"] = fdf["POG 그룹 ID"].astype(int)
final = pd.merge(fdf, pog_in_df, on= "POG 그룹 ID", how = "left")


In [403]:
final.GW_Cate.unique()

array(['입구', '청과', '채소', '닭고기', '수산', '냉장냉동', '조미식품', '데일리', '음료,주류',
       '과자', '위생용품', '건강,차', '세제', '수예', '인테리어', '애완원예용품', '자동차', '계산대',
       '인스턴트', '축산', '델리카', '주방용품', '청소욕실', '완구'], dtype=object)

In [413]:
final.to_csv("동선정보_매핑완료_0122_0220_final.csv", index = False)

In [ ]:
data = final.copy()

In [4]:
data = pd.read_csv('/home/shinjk1156/Project_code/Retail_Project/Code/Preprocessing/Data/동선정보_매핑완료_0122_0220_final.csv')

In [ ]:
data.GW_Cate.unique()

In [5]:
def correction(trans_list):
    count_list = []
    shop_list = []
    i = 0
    j = 1
    count = 1
    while(True):
        if i + j  > len(trans_list)-1:
            count_list.append(count)
            shop_list.append(trans_list[i])
            break
        if trans_list[i] == trans_list[i+j]:
            count += 1
            j +=1
        else:
            count_list.append(count)
            shop_list.append(trans_list[i])
            count = 1
            i = i + j
            j = 1
    return count_list, shop_list
    

In [6]:
def freq(trans_list):
    new_list = []
    new_list.append(trans_list[0])
    for i in range(1, len(trans_list)-1, 1):
        if (trans_list[i] != trans_list[i+1]) and (trans_list[i] != trans_list[i-1]):
            pass
        else:
            new_list.append(trans_list[i])
    new_list.append(trans_list[-1])
    return new_list
        

In [7]:
transactions = []
for id in tqdm(data["쇼핑 ID"].unique()):
    temp = data[data["쇼핑 ID"] == id]
    temp.reset_index(inplace = True, drop = True)
    d_index_list = []
    for i in range(1,len(temp)-1,1):
        if (temp.loc[i,"GW_Cate"] != temp.loc[i+1,"GW_Cate"]) and (temp.loc[i,"GW_Cate"]!= temp.loc[i-1,"GW_Cate"]):
            d_index_list.append(i)
        else:
            pass
    temp.drop(index = d_index_list, inplace = True)
    temp.reset_index(drop = True, inplace  = True)
    cor_list = correction(temp.GW_Cate.to_list()) #횟수, 동선리스트
    i = 0
    f_list = []
    for cor in range(len(cor_list[0])):
        #print(cor_list[1][cor])
        if cor == 0:
            start = datetime.strptime(str(temp.loc[i,"저장시간"]), "%Y-%m-%d %H:%M:%S.%f")
            #print(str(temp.loc[i,"저장시간"]))
            j = cor_list[0][cor]
            i = i + j -1
            end = datetime.strptime(str(temp.loc[i,"저장시간"]), "%Y-%m-%d %H:%M:%S.%f")
            #print(str(temp.loc[i,"저장시간"]))
        else:
            start = datetime.strptime(str(temp.loc[i+1,"저장시간"]), "%Y-%m-%d %H:%M:%S.%f")
            #print(str(temp.loc[i+1,"저장시간"]))
            j = cor_list[0][cor]
            i = i + j 
            end = datetime.strptime(str(temp.loc[i,"저장시간"]), "%Y-%m-%d %H:%M:%S.%f")
            #print(str(temp.loc[i,"저장시간"]))
        f_list = f_list + [cor_list[1][cor]]*(int((end-start).total_seconds()/60) + 1)
    transactions.append(f_list)

100%|██████████| 12506/12506 [01:47<00:00, 115.93it/s]


In [8]:
trans_dic = {}
count = 0
for trans in transactions:
    for element in trans:
        if element not in trans_dic.keys():
            trans_dic[element] = count
            count += 1
        else:
            pass
#동선정보에 기록이 안된 매대(리피터 오류로 추정됨)
out = ["H&B", "곡물","언더웨어","스포츠","문구"]
for h in out:
    trans_dic[h] = count
    count += 1


In [9]:
n_trans_list = []
for trans in transactions:
    t_list = []
    for element in trans:
        t_list.append(str(trans_dic[element]))
    n_trans_list.append(t_list)

In [10]:
trans_matrix = pd.DataFrame(np.zeros((len(trans_dic.keys()), len(trans_dic.keys()))),index = trans_dic.keys(), columns= trans_dic.keys())
#인접지역 입력 "H&B", "곡물","언더웨어","스포츠","문구"
c_state = {"H&B" : ["곡물", "계산대","스포츠","문구","완구","자동차","언더웨어","청소욕실"],
"곡물" : ["청과","계산대","채소","H&B"],
"언더웨어" : ["H&B","애완원예용품","인테리어","건강,차","청소욕실"],
"스포츠" : ["H&B","문구"], "문구" : ["스포츠", "H&B", "완구"]}
#c_state_in =  

In [13]:
for t_list in transactions:
    for t in range(len(t_list)-1):
        trans_matrix.loc[t_list[t], t_list[t+1]] += 1

In [14]:
origin_trans_matrix = trans_matrix.copy()
trans_matrix = origin_trans_matrix.copy()

In [15]:
for state in c_state.keys():
    temp = c_state[state]
    for s in temp:
        trans_matrix.loc[state,s] = 1/len(temp)

In [16]:
for inx in trans_matrix.index:
    #소수점 3번째 이하 0으로 처리
    trans_matrix.loc[inx, :] = round(trans_matrix.loc[inx, :] / sum(trans_matrix.loc[inx, :]),2)
    
    for jix in trans_matrix.index:
        if trans_matrix.loc[inx, jix] < 0.03: 
            trans_matrix.loc[inx, jix] = 0

for state in c_state.keys():
    temp = c_state[state]
    for s in temp:
        if s in ["문구","완구","자동차","애완원예용품","인테리어"]:
            trans_matrix.loc[s,state] = 1/3
        elif s in ["스포츠","청과","계산대"]:
            trans_matrix.loc[s,state] = 1/2
        elif s in ["곡물","청소욕실","건강,차"]:
            trans_matrix.loc[s,state] = 1/4
        elif s == "언더웨어":
            trans_matrix.loc[s,state] = 1/5
        elif s == "채소":
            trans_matrix.loc[s,state] = 1/6
        else:
            trans_matrix.loc[s,state] = 1/8


for inx in trans_matrix.index:
    #다시 각 row의 합이 1이 되게 설정함
    trans_matrix.loc[inx, :] = trans_matrix.loc[inx, :] / sum(trans_matrix.loc[inx, :])


In [538]:
trans_matrix.to_csv("transition_probability.csv")

In [17]:
trans_matrix

,입구,청과,채소,닭고기,수산,냉장냉동,조미식품,데일리,과자,"건강,차",...,델리카,주방용품,청소욕실,위생용품,완구,H&B,곡물,언더웨어,스포츠,문구
입구,0.336842,0.284211,0.378947,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
청과,0.000000,0.202703,0.358108,0.020270,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.337838,0.000000,0.000000,0.000000
채소,0.000000,0.072948,0.382979,0.145897,0.127660,0.000000,0.027356,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.151976,0.000000,0.000000,0.000000
닭고기,0.000000,0.000000,0.791667,0.104167,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
수산,0.000000,0.000000,0.315789,0.000000,0.189474,0.263158,0.115789,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
냉장냉동,0.000000,0.000000,0.000000,0.000000,0.168421,0.273684,0.242105,0.136842,0.000000,0.000000,...,0.126316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
조미식품,0.000000,0.000000,0.043011,0.000000,0.107527,0.290323,0.172043,0.161290,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
데일리,0.000000,0.000000,0.000000,0.000000,0.000000,0.138298,0.148936,0.287234,0.063830,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
과자,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.393617,0.191489,...,0.000000,0.000000,0.000000,0.106383,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"건강,차",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.210526,0.201754,...,0.000000,0.157895,0.000000,0.000000,0.000000,0.000000,0.000000,0.219298,0.000000,0.000000
